<a href="https://colab.research.google.com/github/Ecevangelista/422_Classification_MNIST/blob/main/Assignment_7_2_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import statistics
import numpy as np
from numpy import arange
import statsmodels
import statsmodels.api as sm
from scipy.stats import norm,uniform, reciprocal
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, PrecisionRecallDisplay, f1_score, precision_score, recall_score, classification_report, multilabel_confusion_matrix
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import StandardScaler, scale
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from datetime import datetime

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Ingest

In [2]:
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = "drive/MyDrive/422NU/MNIST_train.csv"
df = pd.read_csv(path)

In [4]:
df1train = df.copy(deep = True)

# EDA and Preprocessing

In [5]:
#split df into test and train
X_train_full = df1train.drop(columns = ['label'])
y_train_full = df1train['label']

In [6]:
#Split train data set into train, validation, test

X_train_fulla, X_testa, y_train_fulla, y_testa = train_test_split(X_train_full, y_train_full, test_size = 0.2, random_state = 10)

X_train, X_valid, y_train, y_valid =train_test_split(X_train_fulla, y_train_fulla, test_size = 0.2, random_state = 10)

In [7]:
X_train.shape

(26880, 784)

In [8]:
X_valid.shape

(6720, 784)

In [9]:
X_testa.shape

(8400, 784)

In [10]:
#scaling the training, validation and test sets to work with keras

X_train = (X_train[:26880]/255.0)
X_valid = (X_valid[:6720]/255.0)
X_test = (X_testa[:8400]/255.0)

# Model 1:  1 layer/100 nodes

In [11]:
model_100a = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=X_train.shape[1:]),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10, activation = "softmax")
])


In [12]:
model_100a.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [13]:
start = datetime.now()

model_100a.fit(X_train, y_train, epochs = 30, 
          validation_data = (X_valid, y_valid))

end = datetime.now()
print("Script run time:", end-start)

Epoch 1/30
840/840 [==============================] - 6s 6ms/step - loss: 0.3741 - accuracy: 0.8952 - val_loss: 0.2478 - val_accuracy: 0.9284
Epoch 2/30
840/840 [==============================] - 3s 4ms/step - loss: 0.1752 - accuracy: 0.9499 - val_loss: 0.1761 - val_accuracy: 0.9513
Epoch 3/30
840/840 [==============================] - 3s 3ms/step - loss: 0.1253 - accuracy: 0.9621 - val_loss: 0.1450 - val_accuracy: 0.9606
Epoch 4/30
840/840 [==============================] - 2s 3ms/step - loss: 0.0940 - accuracy: 0.9732 - val_loss: 0.1339 - val_accuracy: 0.9613
Epoch 5/30
840/840 [==============================] - 3s 3ms/step - loss: 0.0723 - accuracy: 0.9786 - val_loss: 0.1233 - val_accuracy: 0.9655
Epoch 6/30
840/840 [==============================] - 3s 4ms/step - loss: 0.0563 - accuracy: 0.9836 - val_loss: 0.1282 - val_accuracy: 0.9655
Epoch 7/30
840/840 [==============================] - 3s 3ms/step - loss: 0.0454 - accuracy: 0.9867 - val_loss: 0.1178 - val_accuracy: 0.9692
Epoch 

In [14]:
#Evaluate on train set

model_100a.evaluate(X_train, y_train)

840/840 [==============================] - 2s 2ms/step - loss: 4.4952e-04 - accuracy: 1.0000


[0.0004495214088819921, 1.0]

In [15]:
#Evaluate on test set

model_100a.evaluate(X_test, y_testa)

263/263 [==============================] - 1s 4ms/step - loss: 0.1270 - accuracy: 0.9731


[0.12704618275165558, 0.9730952382087708]

In [16]:
#PREDICTIONS

probability_model_100a = tf.keras.Sequential([model_100a, tf.keras.layers.Softmax()])

In [17]:
#GETTING CLASS PREDICTIONS WITH ARGMAX

pred_100a = probability_model_100a.predict(X_test)

classes_100a = np.argmax(pred_100a, axis = 1)

In [18]:
classes_100a[:5]

array([7, 3, 9, 8, 6])

In [19]:
#multiclass confusion matrix 100a
cm100a = confusion_matrix(y_testa, classes_100a)

In [20]:
#create data frame for multi-class confusion matrix 100a

cm100a_df = pd.DataFrame(cm100a, index = [0,1,2,3,4,5,6,7,8,9], columns = [0,1,2,3,4,5,6,7,8,9])
print(cm100a_df)

     0    1    2    3    4    5    6    7    8    9
0  783    0    1    1    1    0    3    1    3    3
1    0  930    3    3    3    0    0    3    4    0
2    3    1  826    6    4    0    5    7    3    1
3    2    0    3  836    0    5    0    1    8    1
4    1    1    2    0  814    0    2    0    0    6
5    1    0    2    5    1  739    8    1   15    1
6    3    1    1    0    1    2  778    0    1    0
7    4    2    6    3    2    2    0  846    0    4
8    5    8    2    6    2    4    7    0  796    6
9    0    0    1    8    9    3    0    7    1  826


In [ ]:
#LOOK AT CLASSIFICATION REPORT

print(classification_report(y_testa, classes_100a))

# Model 2: 1 layer/300 nodes

In [21]:
model_300a = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=X_train.shape[1:]),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(10, activation = "softmax")
])

In [22]:
model_300a.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [23]:
start = datetime.now()

model_300a.fit(X_train, y_train, epochs = 30, 
          validation_data = (X_valid, y_valid))

end = datetime.now()
print("Script run time:", end-start)

Epoch 1/30
840/840 [==============================] - 5s 5ms/step - loss: 0.3034 - accuracy: 0.9133 - val_loss: 0.1809 - val_accuracy: 0.9454
Epoch 2/30
840/840 [==============================] - 4s 5ms/step - loss: 0.1277 - accuracy: 0.9610 - val_loss: 0.1322 - val_accuracy: 0.9628
Epoch 3/30
840/840 [==============================] - 4s 5ms/step - loss: 0.0798 - accuracy: 0.9759 - val_loss: 0.1228 - val_accuracy: 0.9652
Epoch 4/30
840/840 [==============================] - 5s 6ms/step - loss: 0.0559 - accuracy: 0.9832 - val_loss: 0.1210 - val_accuracy: 0.9637
Epoch 5/30
840/840 [==============================] - 5s 6ms/step - loss: 0.0396 - accuracy: 0.9883 - val_loss: 0.1148 - val_accuracy: 0.9686
Epoch 6/30
840/840 [==============================] - 5s 6ms/step - loss: 0.0303 - accuracy: 0.9904 - val_loss: 0.1132 - val_accuracy: 0.9702
Epoch 7/30
840/840 [==============================] - 6s 7ms/step - loss: 0.0215 - accuracy: 0.9941 - val_loss: 0.1291 - val_accuracy: 0.9659
Epoch 

In [ ]:
#Evaluate on train set

model_300a.evaluate(X_train, y_train)

In [ ]:
#Evaluate on test set

model_300a.evaluate(X_test, y_testa)

In [24]:
#PREDICTIONS

probability_model_300a = tf.keras.Sequential([model_300a, tf.keras.layers.Softmax()])

In [25]:
#GETTING CLASS PREDICTIONS WITH ARGMAX

pred_300a = probability_model_300a.predict(X_test)


classes_300a = np.argmax(pred_300a, axis = 1)

In [26]:
classes_300a[:5]

array([7, 3, 9, 8, 6])

In [27]:
#multiclass confusion matrix  300a
cm300a = confusion_matrix(y_testa, classes_300a)

In [28]:
#create data frame for multi-class confusion matrix 300a

cm300a_df = pd.DataFrame(cm300a, index = [0,1,2,3,4,5,6,7,8,9], columns = [0,1,2,3,4,5,6,7,8,9])
print(cm300a_df)

     0    1    2    3    4    5    6    7    8    9
0  787    0    1    1    1    0    2    0    3    1
1    0  937    1    2    2    0    0    3    1    0
2    3    2  839    2    4    0    0    3    2    1
3    2    0    4  834    0    7    0    1    6    2
4    1    1    2    0  819    0    1    0    1    1
5    0    0    3    6    0  753    4    1    4    2
6    3    2    0    0    2    2  778    0    0    0
7    2    2    6    2    1    3    0  851    0    2
8    2    9    3    6    3    1    3    2  805    2
9    0    0    1    4    4    3    0    6    1  836


In [ ]:
#LOOK AT CLASSIFICATION REPORT

print(classification_report(y_testa, classes_300a))

# Model 3: 2 layers/100 nodes

In [29]:
model_1002b = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=X_train.shape[1:]),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10, activation = "softmax")
])

In [30]:
model_1002b.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [31]:
start = datetime.now()

model_1002b.fit(X_train, y_train, epochs = 30, 
          validation_data = (X_valid, y_valid))

end = datetime.now()
print("Script run time:", end-start)

Epoch 1/30
840/840 [==============================] - 4s 4ms/step - loss: 0.3413 - accuracy: 0.8982 - val_loss: 0.1961 - val_accuracy: 0.9418
Epoch 2/30
840/840 [==============================] - 4s 4ms/step - loss: 0.1459 - accuracy: 0.9568 - val_loss: 0.1456 - val_accuracy: 0.9589
Epoch 3/30
840/840 [==============================] - 5s 6ms/step - loss: 0.1016 - accuracy: 0.9685 - val_loss: 0.1377 - val_accuracy: 0.9604
Epoch 4/30
840/840 [==============================] - 4s 5ms/step - loss: 0.0741 - accuracy: 0.9774 - val_loss: 0.1302 - val_accuracy: 0.9632
Epoch 5/30
840/840 [==============================] - 3s 3ms/step - loss: 0.0537 - accuracy: 0.9829 - val_loss: 0.1251 - val_accuracy: 0.9656
Epoch 6/30
840/840 [==============================] - 4s 4ms/step - loss: 0.0417 - accuracy: 0.9873 - val_loss: 0.1384 - val_accuracy: 0.9634
Epoch 7/30
840/840 [==============================] - 5s 5ms/step - loss: 0.0335 - accuracy: 0.9889 - val_loss: 0.1474 - val_accuracy: 0.9655
Epoch 

In [ ]:
#Evaluate on train set

model_1002b.evaluate(X_train, y_train)

In [ ]:
#Evaluate on test set

model_1002b.evaluate(X_test, y_testa)

In [32]:
#PREDICTIONS

probability_model_1002b = tf.keras.Sequential([model_1002b, tf.keras.layers.Softmax()])

In [33]:
#GETTING CLASS PREDICTIONS WITH ARGMAX

pred_1002b = probability_model_1002b.predict(X_test)

classes_1002b = np.argmax(pred_1002b, axis = 1)

In [34]:
classes_1002b[:5]

array([7, 3, 9, 8, 6])

In [35]:
#multiclass confusion matrix 1002b
cm1002b = confusion_matrix(y_testa, classes_1002b)

In [36]:
#create data frame for multi-class confusion matrix 1002b

cm1002b_df = pd.DataFrame(cm1002b, index = [0,1,2,3,4,5,6,7,8,9], columns = [0,1,2,3,4,5,6,7,8,9])
print(cm1002b_df)

     0    1    2    3    4    5    6    7    8    9
0  777    1    4    1    1    0    7    0    3    2
1    0  926   10    1    1    0    0    4    4    0
2    3    1  838    0    2    0    3    4    2    3
3    2    0   10  825    0   10    1    1    5    2
4    1    1    6    0  801    0    3    1    1   12
5    1    1    1   11    0  744    7    1    6    1
6    4    0    1    0    2    0  779    0    1    0
7    1    1   13    4    1    3    0  830    3   13
8    2    5   17    5    1    5    6    4  778   13
9    0    0    1    6    2    2    0    3    2  839


In [ ]:
#LOOK AT CLASSIFICATION REPORT

print(classification_report(y_testa, classes_1002b))

# Model 4: 2 layers/300 nodes

In [37]:
model_3002b = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=X_train.shape[1:]),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(10, activation = "softmax")
])

In [38]:
model_3002b.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [39]:
start = datetime.now()

model_3002b.fit(X_train, y_train, epochs = 30, 
          validation_data = (X_valid, y_valid))

end = datetime.now()
print("Script run time:", end-start)

Epoch 1/30
840/840 [==============================] - 6s 7ms/step - loss: 0.2671 - accuracy: 0.9187 - val_loss: 0.1706 - val_accuracy: 0.9499
Epoch 2/30
840/840 [==============================] - 5s 6ms/step - loss: 0.1058 - accuracy: 0.9670 - val_loss: 0.1282 - val_accuracy: 0.9638
Epoch 3/30
840/840 [==============================] - 8s 10ms/step - loss: 0.0673 - accuracy: 0.9781 - val_loss: 0.1187 - val_accuracy: 0.9676
Epoch 4/30
840/840 [==============================] - 7s 9ms/step - loss: 0.0487 - accuracy: 0.9848 - val_loss: 0.1280 - val_accuracy: 0.9667
Epoch 5/30
840/840 [==============================] - 7s 9ms/step - loss: 0.0359 - accuracy: 0.9881 - val_loss: 0.1356 - val_accuracy: 0.9667
Epoch 6/30
840/840 [==============================] - 7s 9ms/step - loss: 0.0329 - accuracy: 0.9891 - val_loss: 0.1341 - val_accuracy: 0.9701
Epoch 7/30
840/840 [==============================] - 5s 6ms/step - loss: 0.0265 - accuracy: 0.9912 - val_loss: 0.1529 - val_accuracy: 0.9649
Epoch

In [52]:
#Evaluate on train set

model_3002b.evaluate(X_train, y_train)

840/840 [==============================] - 2s 2ms/step - loss: 0.0033 - accuracy: 0.9988


[0.003275353228673339, 0.998846709728241]

In [53]:
#Evaluate on test set

model_3002b.evaluate(X_test, y_testa)

263/263 [==============================] - 1s 3ms/step - loss: 0.1682 - accuracy: 0.9760


[0.16817156970500946, 0.9759523868560791]

In [40]:
#PREDICTIONS

probability_model_3002b = tf.keras.Sequential([model_3002b, tf.keras.layers.Softmax()])

In [41]:
#GETTING CLASS PREDICTIONS WITH ARGMAX

pred_3002b = probability_model_3002b.predict(X_test)

classes_3002b = np.argmax(pred_3002b, axis = 1)

In [42]:
classes_3002b[:5]

array([7, 3, 9, 8, 6])

In [43]:
#multiclass confusion matrix 300 2b
cm3002b = confusion_matrix(y_testa, classes_3002b)

In [44]:
#create data frame for multi-class confusion matrix 3002b

cm3002b_df = pd.DataFrame(cm3002b, index = [0,1,2,3,4,5,6,7,8,9], columns = [0,1,2,3,4,5,6,7,8,9])
print(cm3002b_df)

     0    1    2    3    4    5    6    7    8    9
0  784    1    1    1    0    0    4    0    3    2
1    0  936    4    0    2    0    0    3    1    0
2    1    2  837    1    5    0    1    5    3    1
3    2    0    5  827    1   14    0    1    5    1
4    0    2    1    0  817    1    3    0    0    2
5    0    0    2    2    0  760    5    0    3    1
6    3    1    0    0    2    3  778    0    0    0
7    1    1   13    2    3    2    1  842    1    3
8    1    9    3    3    4    4    6    1  804    1
9    0    0    2    5   19    9    0    5    2  813


In [54]:
#LOOK AT CLASSIFICATION REPORT

print(classification_report(y_testa, classes_3002b))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99       796
           1       0.98      0.99      0.99       946
           2       0.96      0.98      0.97       856
           3       0.98      0.97      0.97       856
           4       0.96      0.99      0.97       826
           5       0.96      0.98      0.97       773
           6       0.97      0.99      0.98       787
           7       0.98      0.97      0.98       869
           8       0.98      0.96      0.97       836
           9       0.99      0.95      0.97       855

    accuracy                           0.98      8400
   macro avg       0.98      0.98      0.98      8400
weighted avg       0.98      0.98      0.98      8400



# Model 5: 2 layers/300 nodes/ 20% dropout

In [45]:
dropout_model2 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=X_train.shape[1:]),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax'),
])

In [46]:
dropout_model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [47]:
start = datetime.now()

dropout_model2.fit(X_train, y_train, epochs = 30, 
          validation_data = (X_valid, y_valid))

end = datetime.now()
print("Script run time:", end-start)

Epoch 1/30
840/840 [==============================] - 6s 6ms/step - loss: 0.3288 - accuracy: 0.8991 - val_loss: 0.1716 - val_accuracy: 0.9470
Epoch 2/30
840/840 [==============================] - 5s 6ms/step - loss: 0.1406 - accuracy: 0.9561 - val_loss: 0.1484 - val_accuracy: 0.9555
Epoch 3/30
840/840 [==============================] - 7s 8ms/step - loss: 0.1052 - accuracy: 0.9661 - val_loss: 0.1277 - val_accuracy: 0.9634
Epoch 4/30
840/840 [==============================] - 9s 11ms/step - loss: 0.0799 - accuracy: 0.9733 - val_loss: 0.1243 - val_accuracy: 0.9650
Epoch 5/30
840/840 [==============================] - 6s 7ms/step - loss: 0.0667 - accuracy: 0.9797 - val_loss: 0.1131 - val_accuracy: 0.9674
Epoch 6/30
840/840 [==============================] - 8s 9ms/step - loss: 0.0600 - accuracy: 0.9813 - val_loss: 0.1266 - val_accuracy: 0.9665
Epoch 7/30
840/840 [==============================] - 8s 9ms/step - loss: 0.0536 - accuracy: 0.9827 - val_loss: 0.1209 - val_accuracy: 0.9704
Epoch

In [ ]:
#Evaluate on train set

dropout_model2.evaluate(X_train, y_train)

In [ ]:
#Evaluate on test set

dropout_model2.evaluate(X_test, y_testa)

In [48]:
#PREDICTIONS

probability_model_drop2 = tf.keras.Sequential([dropout_model2, tf.keras.layers.Softmax()])

In [49]:
#GETTING CLASS PREDICTIONS WITH ARGMAX

pred_drop2 = probability_model_drop2.predict(X_test)

classes_drop2 = np.argmax(pred_drop2, axis = 1)

In [50]:
#multiclass confusion matrix with dropout 2

cmdrop2 = confusion_matrix(y_testa, classes_drop2)

In [51]:
#create data frame for multi-class confusion matrix drop2

cmdrop2_df = pd.DataFrame(cmdrop2, index = [0,1,2,3,4,5,6,7,8,9], columns = [0,1,2,3,4,5,6,7,8,9])
print(cmdrop2_df)

     0    1    2    3    4    5    6    7    8    9
0  783    1    1    0    0    0    4    2    2    3
1    0  937    0    2    2    0    0    1    4    0
2    3    5  834    2    4    0    3    2    3    0
3    0    0    9  830    0    9    0    1    3    4
4    1    1    1    0  820    0    3    0    0    0
5    2    0    1    3    0  749    6    2    9    1
6    1    3    0    0    3    1  778    0    1    0
7    2    2   16    1    2    2    0  842    0    2
8    0    7    6    3    2    4    6    2  801    5
9    1    0    0    2   10    2    0    6    2  832


In [ ]:
#LOOK AT CLASSIFICATION REPORT

print(classification_report(y_testa, classes_drop2))

# Conclusion

Model 2 with 1 layer/300 nodes achieved the highest testing accuracy (97.9%) and F1 score (0.98) performance. Applying dropout layers to the worst-performing model (Model 4) boosted performance to closely match Model 2, by producing a testing accuracy of 97.7% and F1 score of 0.98.